# 评分卡建模

## 信用卡数据

**导入依赖包**

In [75]:
#载入相关包
import os
# import lightgbm as lgb
import numpy as np
import pandas as pd
import scorecardpy as sc
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import sys
pd.set_option('display.unicode.ambiguous_as_wide',True)
pd.set_option('display.unicode.east_asian_width',True)
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',10000)
import statsmodels.api as sm
import  LuckyDer as lk 
from imp import reload
reload(lk)
from LuckyDer import *
# from openpyxl import Workbook
# from openpyxl.utils.dataframe import dataframe_to_rows


<module 'LuckyDer' from '/Users/gengbh/code/vscode/Data-Analysis-Notes/评分卡建模/LuckyDer.py'>

**数据预处理**

In [12]:
dat=pd.read_csv(u'/Users/gengbh/code/vscode/Data-Analysis-Notes/评分卡建模/day08_rankingcard.csv')

In [13]:
dat.shape

(150000, 12)

In [14]:
dat.columns

Index(['Unnamed: 0', 'SeriousDlqin2yrs',
       'RevolvingUtilizationOfUnsecuredLines', 'age',
       'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
       'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
       'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
       'NumberOfDependents'],
      dtype='object')

In [15]:
dat=dat.iloc[:,1:]

In [16]:
dat.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


In [17]:
dat.isnull().sum()/dat.shape[0]

SeriousDlqin2yrs                        0.000000
RevolvingUtilizationOfUnsecuredLines    0.000000
age                                     0.000000
NumberOfTime30-59DaysPastDueNotWorse    0.000000
DebtRatio                               0.000000
MonthlyIncome                           0.198207
NumberOfOpenCreditLinesAndLoans         0.000000
NumberOfTimes90DaysLate                 0.000000
NumberRealEstateLoansOrLines            0.000000
NumberOfTime60-89DaysPastDueNotWorse    0.000000
NumberOfDependents                      0.026160
dtype: float64

In [18]:
#去除重复值
dat.drop_duplicates(inplace=True)
# 缺失值处理 使用均值填充
dat["NumberOfDependents"].fillna(int(dat["NumberOfDependents"].mean()),inplace=True)
# dat.info()
# 恢复索引
dat.index = range(dat.shape[0])

In [19]:
def fill_missing_rf(X,y,to_fill):

    """
    使用随机森林填补一个特征的缺失值的函数

    参数：
    X：要填补的特征矩阵
    y：完整的，没有缺失值的标签
    to_fill：字符串，要填补的那一列的名称
    """

    #构建我们的新特征矩阵和新标签
    df = X.copy()
    fill = df.loc[:,to_fill]
    df = pd.concat([df.loc[:,df.columns != to_fill],pd.DataFrame(y)],axis=1)

    # 找出我们的训练集和测试集
    Ytrain = fill[fill.notnull()]
    Ytest = fill[fill.isnull()]
    Xtrain = df.iloc[Ytrain.index,:]
    Xtest = df.iloc[Ytest.index,:]

    #用随机森林回归来填补缺失值
    from sklearn.ensemble import RandomForestRegressor as rfr
    rfr = rfr(n_estimators=100)
    rfr = rfr.fit(Xtrain, Ytrain)
    Ypredict = rfr.predict(Xtest)

    return Ypredict

In [20]:
X = dat.iloc[:,1:]
y = dat["SeriousDlqin2yrs"]#y = data.iloc[:,0]
y_pred = fill_missing_rf(X,y,"MonthlyIncome")

#注意可以通过以下代码检验数据是否数量相同
# y_pred.shape ==  data.loc[data.loc[:,"MonthlyIncome"].isnull(),"MonthlyIncome"].shape
#确认我们的结果合理之后，我们就可以将数据覆盖了a
dat.loc[dat.loc[:,"MonthlyIncome"].isnull(),"MonthlyIncome"] = y_pred
# data.info()

In [26]:
# 将处理好的数据存储为csv
dat.to_csv('dat.csv')


## 数据准备至此完成

In [56]:
dat1 = pd.read_csv('dat.csv')
path=os.getcwd()

In [57]:
#划分训练测试集
dat_train,dat_test = train_test_split(dat,test_size = 0.3,random_state = 886)
# 恢复索引
dat_train = train.sort_index()
dat_test = test.sort_index()

In [66]:
print('训练集数据量：\n {}'.format(dat_train.shape))
print('测试集数据量：\n {}'.format(dat_test.shape))

训练集数据量：
 (104573, 11)
测试集数据量：
 (44818, 11)


## 特征筛选

**缺失率&集中度筛选**

In [73]:
# #### 2.1 缺失值集中度筛选
def feature_selector(dat,keep_col,nouse_col):
    #删除日期变量
    drop_col_for_dt = reg(".+dt",dat.columns)
    dat = dat.drop(drop_col_for_dt,axis = 1)
    print("删除日期变量：{}".format(drop_col_for_dt))
    print(dat.shape)
    
    #删除date变量
    drop_col_for_date = reg(".+date",dat.columns)
    dat = dat.drop(drop_col_for_date,axis = 1)
    print("删除date变量：{}".format(drop_col_for_date))
    print(dat.shape)
    
    #删除cid 和 cd_cid,无用Y变量
    dat = dat.drop([i for i in ['cid'] if i in dat.columns],axis = 1)

    ##自定义集中度函数
    def group_cnt(x):
        try:
            res = x.value_counts(normalize =True).sort_values(ascending =  False).tolist()[0]
            return res
        except:
            #print(x.name)
            return 0
    x = dat.apply(group_cnt)
    #删除集中度>95的变量，保留历史购买保险
    drop_col_for_focu = [i for i in x[x>=0.95].index.tolist() ]
    dat = dat.drop(drop_col_for_focu,axis = 1)
    print("删除集中度>95的变量，保留历史购买保险：{}".format(drop_col_for_focu))
    print(dat.shape)

    #剔除事后变量
    drop_col_for_After = [i for i in dat.columns if  i in reg("v1.+",dat.columns) + ['effective','mon','starttime','endtime','starttime_endtime','callcount','refusereason']]
    dat = dat.drop(drop_col_for_After,axis = 1)
    print("剔除事后变量：{}".format(drop_col_for_After))
    print(dat.shape)

    #剔除type变量
    drop_col_for_type = [i for i in dat.columns if  i in reg(".+type$",dat.columns)]
    dat = dat.drop(drop_col_for_type,axis = 1)
    print("剔除type变量：{}".format(drop_col_for_type))
    print(dat.shape)
    
    #干掉类别变量
    drop_col_for_class = [i for i in dat.select_dtypes('object').columns.tolist() if i not in ['age_gender','age_marriage', 'gender_marriage','gender_education', 'marriage_education']]
    dat = dat.drop(drop_col_for_class,axis = 1)
    print("干掉类别变量：{}".format(drop_col_for_class))
    print(dat.shape)
    
    #剔除缺失率>95的变量，保留保险和信用卡用信率字段
    miss_rate = miss(dat[dat.columns[~dat.columns.isin(reg('buy.+|user_rate.+',dat.columns))].tolist()])
    drop_col_for_miss = miss_rate[miss_rate.miss_rate >= 0.95].index.tolist()
    dat = dat.drop(drop_col_for_miss,axis = 1)
    print("干掉缺失95变量：{}".format(drop_col_for_miss))
    print(dat.shape)
    dropcol_for_dt = pd.DataFrame(drop_col_for_dt,columns = ['drop_col'])
    dropcol_for_dt['DropReason'] = 'dt'
    dropcol_for_dt['Count'] = len(drop_col_for_dt)

    dropcol_for_date = pd.DataFrame(drop_col_for_date,columns = ['drop_col'])
    dropcol_for_date['DropReason'] = 'date'
    dropcol_for_date['Count'] = len(drop_col_for_date)

    dropcol_for_focu = pd.DataFrame(drop_col_for_focu,columns = ['drop_col'])
    dropcol_for_focu['DropReason'] = 'focu'
    dropcol_for_focu['Count'] = len(drop_col_for_focu)

    dropcol_for_After = pd.DataFrame(drop_col_for_After,columns = ['drop_col'])
    dropcol_for_After['DropReason'] = 'after'
    dropcol_for_After['Count'] = len(drop_col_for_After)

    dropcol_for_type = pd.DataFrame(drop_col_for_type,columns = ['drop_col'])
    dropcol_for_type['DropReason'] = 'type'
    dropcol_for_type['Count'] = len(drop_col_for_type)

    dropcol_for_class = pd.DataFrame(drop_col_for_class,columns = ['drop_col'])
    dropcol_for_class['DropReason'] = 'class'
    dropcol_for_class['Count'] = len(drop_col_for_class)

    dropcol_for_miss = pd.DataFrame(drop_col_for_miss,columns = ['drop_col'])
    dropcol_for_miss['DropReason'] = 'miss'
    dropcol_for_miss['Count'] = len(drop_col_for_miss)
    dropcol_reason = pd.concat([dropcol_for_dt,dropcol_for_date,dropcol_for_focu,dropcol_for_After, dropcol_for_type,dropcol_for_class,dropcol_for_miss])
    dropcol_reason.to_csv(path + '//dropcol_reason.csv')
    return dat,dropcol_reason
def cate_trans(dat):
    #类别变量转换
    #类别转换规则，根据坏账排序
    dat = dat.copy()
    a,b = cate_var_transform(dat.select_dtypes('object'),dat['flagy'])
        #转换结果
    c = {}
    obj = dat.select_dtypes('object').columns.tolist()
    for i in range(0,len(obj),1): #循环类别变量
        di = {}
        for j in range(0,len(b[i]),1): #循环每行取值
            di[b[i]['raw data'][j]] = b[i]['transform data'][j]  #将每行结果转换dict
        dat[obj[i]] = dat[obj[i]].replace(di)
        c[obj[i]] = di
    trans_rule = b
    print(trans_rule)
    print(c)
    return dat,c


In [76]:
#变量筛选
dt1,dropcol_reason = feature_selector(dat1)
dt1,trans_rule = cate_trans(dt1)


删除日期变量：[]
(149391, 12)
删除date变量：[]
(149391, 12)
删除集中度>95的变量，保留历史购买保险：[]
(149391, 12)
剔除事后变量：[]
(149391, 12)
剔除type变量：[]
(149391, 12)
干掉类别变量：[]
(149391, 12)
干掉缺失95变量：[]
(149391, 12)


KeyError: 'flagy'

In [69]:
# #### 2.2  聚类、Lasso、IV筛选
#1)IV筛选
def feature_filter_iv(train,test,flagy):    
    train = train.copy()
    test = test.copy()
    #分箱
    bins = sc.woebin(train,y = flagy)
   
    #woe转换
    train_woe  = sc.woebin_ply(train,bins = bins)
    test_woe  = sc.woebin_ply(test,bins = bins)

    iv_info = iv(train_woe,y = flagy)
    
    ####################add:模型文档：变量IV#####################
    iv_info_f1 = iv_info.sort_values(by = 'info_value',ascending = False)
    iv_info_f1['info_value'] = iv_info_f1.info_value.round(4)
    iv_info_f1['variable'] = [i.replace("_woe",'') for i in iv_info_f1.variable.tolist()]
    ##计算缺失率
    tmp = miss(train[iv_info_f1.variable])
    iv_info_f1['miss_rate'] = tmp['miss_rate'].tolist()

    #数据输出
    df_iv = pd.DataFrame(columns = ['number','variable','exp','IV','miss_rate'])
    df_iv['number'] = list(range(1,iv_info_f1.shape[0]+1))
    df_iv['variable'] = iv_info_f1['variable'].tolist()
    df_iv['IV'] = iv_info_f1['info_value'].tolist()
    df_iv['miss_rate'] = iv_info_f1['miss_rate'].tolist()
    df_iv.to_csv(path + '//df_iv.csv')
    
    #########################存储删除的变量###################################
    drop_col_for_iv = [i for i in iv_info[iv_info.info_value <= 0.02].variable.tolist()]
    train_woe = train_woe.drop(drop_col_for_iv,axis = 1)
    test_woe = test_woe.drop(drop_col_for_iv,axis = 1)
    #dat = dat.drop(drop_col,axis = 1)
    print("iv剔除变量：{}".format(drop_col_for_iv))
    print(train_woe.shape)
    
    dropcol_for_iv = pd.DataFrame(drop_col_for_iv,columns = ['drop_col'])
    dropcol_for_iv['DropReason'] = 'iv'
    dropcol_for_iv['Count'] = len(drop_col_for_iv)
    
    return train_woe,test_woe,bins,dropcol_for_iv,df_iv

train_woe,test_woe,bins,dropcol_for_iv,df_iv= feature_filter_iv(dat_train,dat_test,'SeriousDlqin2yrs')

dropcol_reason = pd.concat([dropcol_reason,dropcol_for_iv])
dropcol_reason.to_csv(path + '//dropcol_reason.csv')


[INFO] creating woe binning ...
[INFO] converting into woe values ...
[INFO] converting into woe values ...
NumberOfTime30-59DaysPastDueNotWorse_woe
还剩变量：9
NumberOfTime60-89DaysPastDueNotWorse_woe
还剩变量：8
NumberOfOpenCreditLinesAndLoans_woe
还剩变量：7
age_woe
还剩变量：6
NumberOfTimes90DaysLate_woe
还剩变量：5
NumberOfDependents_woe
还剩变量：4
MonthlyIncome_woe
还剩变量：3
NumberRealEstateLoansOrLines_woe
还剩变量：2
RevolvingUtilizationOfUnsecuredLines_woe
还剩变量：1
DebtRatio_woe
还剩变量：0
iv剔除变量：[]
(104573, 11)


NameError: name 'dropcol_reason' is not defined

In [54]:

#2）LASSO+聚类筛选
def feature_selector_als(train_woe):
    train_woe = train_woe.copy()

    #删除psi不稳定的变量
    #psi_df,drop_psi,keep_psi = psi(train_woe,test_woe,0.2)
    #drop_psi
    
    print("#####LASSO 筛选....#####")
    #lasso，逻辑回归中的l1
    from sklearn.linear_model import LogisticRegression
    reg_x = train_woe.drop(['flagy'],axis = 1)
    reg_y = train_woe['flagy']
    from sklearn.linear_model import LogisticRegression# lr的Lasso模型
    logis = LogisticRegression(
       penalty = 'l1'
        ,C = 0.1
        ,solver ='liblinear').fit(X =reg_x ,y = reg_y)
    #系数
    coef = pd.Series(logis.coef_[0], index = reg_x.columns)   
    print("Lasso picked " + str(sum(coef > 0)) + 
          " variables and eliminated the other " +  
          str(sum(coef <= 0)) + " variables")   
    print("Lasso变量个数:")
    print(np.sum(logis.coef_ > 0))
    #最终lasso结果变量
    import  matplotlib as plt
    imp_coef = pd.concat([coef.sort_values().head(10),
                         coef.sort_values().tail(10)])
    plt.rcParams['figure.figsize'] = (8.0, 10.0)
    imp_coef.plot(kind = "barh")
    plt.pyplot.title("Coefficients in the Lasso Model")
    #最终lasso后的变量
    lasso_lst = [i for i in list(coef.index[coef.values>0]) + ['flagy']]
    
    ###########################add:存储因lasso删除的变量########################
    drop_col_for_lasso = list(set(train_woe.columns.tolist())-set(lasso_lst))
    
    
    train_woe = train_woe[lasso_lst]
    print("lasso变量：{}".format(lasso_lst))
    print(train_woe.shape)
    
    print("##### 聚类筛选..########")
    def varclus(train_woe):
        from  varclushi import VarClusHi
        #聚类
        vc = VarClusHi(train_woe,maxeigval2=1,maxclus=None).varclus()
        #计算iv
        iv_info = iv(train_woe,y = 'flagy')
        #合并
        d = pd.merge(left = vc.rsquare,right = iv_info,left_on= 'Variable',right_on='variable')
        #每个类寻找那红iv最大的行
        d['rw'] = d.groupby(['Cluster'])['info_value'].rank(method='max', ascending=False, na_option='keep', pct=False, axis=0)
        return d
    vc = varclus(train_woe)
    model_list = list(vc[vc.rw < 3].variable) +['flagy']

    ###########################add:存储因聚类删除的变量########################
    drop_col_for_clus = list(set(train_woe.columns.tolist())-set(model_list))
    
    train_woe = train_woe[model_list]
    print("聚类后变量：{}".format(model_list))
    print(train_woe.shape)
    
    #######汇总所有删除的变量
    dropcol_for_lasso = pd.DataFrame(drop_col_for_lasso,columns = ['drop_col'])
    dropcol_for_lasso['DropReason'] = 'lasso'
    dropcol_for_lasso['Count'] = len(drop_col_for_lasso)

    dropcol_for_clus = pd.DataFrame(drop_col_for_clus,columns = ['drop_col'])
    dropcol_for_clus['DropReason'] = 'clus'
    dropcol_for_clus['Count'] = len(drop_col_for_clus)
    
    dropcol_reason1 = pd.concat([dropcol_for_lasso,dropcol_for_clus])
    return train_woe,vc,dropcol_reason1

In [ ]:
train_woe,vc,dropcol_reason1 = feature_selector_als(train_woe)